In [1]:
import pandas as pd
import numpy as np
# from gensim.models import Word2Vec
# from gensim.models.doc2vec import TaggedDocument
import json
from sklearn.model_selection import train_test_split
import string as string
import time
import re
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import mutual_info_classif
from nltk.tokenize import word_tokenize

from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC

from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import make_scorer

In [2]:
doc = pd.read_csv('data/keluhan_down.csv', encoding='ISO-8859-1')
# doc['Keluhan'] = doc['Keluhan'].map({'Ya':1}).fillna(0)
# doc['Bukan Keluhan/Respon'] = doc['Bukan Keluhan/Respon'].map({'Ya':1}).fillna(0)
# doc['Respon'] = doc['Respon'].map({'Ya':'1'}).fillna(0)
X = doc['Tweet']
doc.head()

,ID,Timestamp,Tweet,Keluhan,Respon,Bukan Keluhan/Respon,Topik Umum,Topik Spesifik,Lokasi,Waktu
0,6.790000e+17,"21 Des 2015, 02:50:25 PM",@EL_Atheos @ridwankamil ya mungkin karena pere...,NaN,NaN,Ya,Bukan Keluhan,-,-,-
1,6.790000e+17,"21 Des 2015, 02:53:02 PM",@ridwankamil @dbmpkotabdg kang teman saya tert...,Ya,NaN,NaN,Lingkungan Hidup,pohon tumbang,Jalan Sangkuriang depan Polsek Coblong,-
2,6.790000e+17,"21 Des 2015, 02:57:55 PM",Di tribun jabar biasanya suka di post agenda k...,NaN,NaN,Ya,Bukan Keluhan,-,-,-
3,6.790000e+17,"21 Des 2015, 02:58:54 PM",@dbmpkotabdg RT @fajriattack: Lapor pak @ridwa...,Ya,NaN,NaN,Infrastruktur,lampu penerangan jalan umum,Jalan depan Kampus LPKIA,-
4,6.790000e+17,"21 Des 2015, 03:06:49 PM","@diskamtam bapak/ibu mau tanya, kalo pemelihar...",NaN,NaN,Ya,Bukan Keluhan,-,-,-


In [3]:
data = doc.loc[:, 'Tweet':'Bukan Keluhan/Respon']
temp = doc['Keluhan'].values.tolist()
idx_keluhan = [i for i, x in enumerate(temp) if x == 'Ya']

temp = doc['Respon'].values.tolist()
idx_respon = [i for i, x in enumerate(temp) if x == 'Ya']

temp = doc['Bukan Keluhan/Respon'].values.tolist()
idx_notboth = [i for i, x in enumerate(temp) if x == 'Ya']

# create 1 column for label
y = []
for i in range (0,len(data)):
    if i in idx_keluhan:
        y.append(0)
    elif i in idx_respon:
        y.append(1)
    else:
        y.append(2)

In [4]:
def removeURL(raw):
    cleanr = re.compile("http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+")
    cleantext = re.sub(cleanr, '', raw)
    
    return cleantext

preprocess = []

for i in range(len(X)):
    preprocess.append(removeURL(X[i]))

X = preprocess

X[1]

u'@ridwankamil @dbmpkotabdg kang teman saya tertimpa pohn dijln sangkuriang dpn polsek coblong tlg ditertibkan phn yg sdh lapuknuhun'

In [5]:
def removeTag(raw):
    cleanr = re.compile("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)")
    cleantext = re.sub(cleanr, '', raw)
    
    return cleantext

preprocess = []

for i in range(len(X)):
    preprocess.append(removeTag(X[i]))

X = preprocess

X[1]

u'  kang teman saya tertimpa pohn dijln sangkuriang dpn polsek coblong tlg ditertibkan phn yg sdh lapuknuhun'

In [6]:
def removedPunctutation(text):
    removed = set(string.punctuation)
    
    return ''.join(w for w in text if w not in removed)

preprocess = []

for i in range(len(X)):
    preprocess.append(removedPunctutation(X[i]))

X = preprocess
X[1]

u'  kang teman saya tertimpa pohn dijln sangkuriang dpn polsek coblong tlg ditertibkan phn yg sdh lapuknuhun'

In [7]:
def removeStopWord(text):
    factory = StopWordRemoverFactory()
    stopword = factory.create_stop_word_remover()
    return stopword.remove(text)

preprocess = []

for i in range(len(X)):
    preprocess.append(removeStopWord(X[i]))

X = preprocess
X[1]

u'  kang teman tertimpa pohn dijln sangkuriang dpn polsek coblong tlg ditertibkan phn yg sdh lapuknuhun'

In [8]:
# def stemWord(text):
#     factory = StemmerFactory()
#     stemmer = factory.create_stemmer()
    
#     return stemmer.stem(text)

# preprocess = []

# for i in range(len(X)):
#     preprocess.append(stemWord(X[i]))

# X = preprocess
# X[1]

In [9]:
def stemWord(w):
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()
    
    words = word_tokenize(w)
    
    sentence = ''
    for word in words:
        sentence += str(stemmer.stem(word)) + ' '
    return sentence

def stemming(X):
    preprocess1 = []

    for i in range(len(X)):
        preprocess1.append(stemWord(X[i]))
    
    return preprocess1

In [10]:
# data_stem = stemming(X)

# Ekstraksi Fitur

In [43]:
count_vect = CountVectorizer()
count_vect.fit(X)
x = count_vect.transform(X)

In [12]:
# Y=X
# x_vec = count_vect.transform(Y)
# x = mutual_info_classif(x_vec, y, discrete_features=True)

In [27]:
X

[u'Atheos  mungkin perempuan bj merah yg paling kelihatan berada depan laki2 jg sering jadi bahan joke RK',
 u'  kang teman tertimpa pohn dijln sangkuriang dpn polsek coblong tlg ditertibkan phn yg sdh lapuknuhun',
 u'Di tribun jabar biasanya suka post agenda kang emil Tapi gak hari sih RT ory Klo k Bdg bs ketemu Kang  dmn',
 u' RT  Lapor pak  lampu penerangan jalan depan kampus lpkia ngaplek  ',
 u' bapakibu mau tanya kalo pemeliharaan taman2 yg banyak dibagun skr gimana nantinya',
 u'Pa  tempat kerja tgl 25 tetep masuk gimana aturannya pa kalo misalnya ga masuk dipotong gaji ',
 u'  00106700466 punten seminggu air PDAM mengalir Mohon bantuannya segera',
 u'RT  Anak SMA ketahuan mencuri helem bip hari pak  ',
 u' ternyata warga yg konfirm pdamkatanya air akan mengalir selama 1 bulan krn ada perbaikan ',
 u'Kang  jalan Muararajeun Baru baru sebulanan dibenerinudah rusak Yang ngerjain kaya ga serius ',
 u'RT  Didinyamah kieu  bosss Kumaha pak  daerah sapharuhai yeuh kang parkir liar see

In [51]:
x

<5076x11404 sparse matrix of type '<type 'numpy.int64'>'
	with 61459 stored elements in Compressed Sparse Row format>

In [55]:
from sklearn.externals import joblib
joblib.dump(count_vect, 'vectorizer.joblib')

['vectorizer.joblib']

In [38]:
tweet = '@ridwankamil @dbmpkotabdg kang teman saya tertimpa pohn dijln sangkuriang dpn polsek coblong tlg ditertibkan phn yg sdh lapuknuhun'
pre_tweet = stemWord(removeStopWord(removeTag(removeURL(tweet.encode('utf-8')))))
pre_tweet

'kang teman timpa pohn dijln sangkuriang dpn polsek coblong tlg tertib phn yg sdh lapuknuhun '

In [56]:
# count_vect = CountVectorizer()
# count_vect.fit(pre_tweet)
count_vect = joblib.load('vectorizer.joblib')
vec = count_vect.transform([pre_tweet])

In [57]:
pred = svc.predict(vec)
pred[0]

0

In [13]:
print(len(X))
# print(x_vec.shape[0])
print(x.shape[0])
print(len(y))

5076
5076
5076


In [14]:
# vectorizer = TfidfVectorizer()
# x = vectorizer.fit_transform(X) 

In [15]:
text_train, text_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [16]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=42)

In [17]:
neigh = KNeighborsClassifier(n_neighbors=3)

neigh.fit(x_train, y_train)

neigh_pred = neigh.predict(x_test)
print('F1 Score KNN : ', f1_score(neigh_pred, y_test, average='macro'))
print('Accuracy Score KNN : ', accuracy_score(neigh_pred, y_test))
print('Precision Score KNN : ', precision_score(neigh_pred, y_test, average='macro'))
print('Recall Score KNN : ', recall_score(neigh_pred, y_test, average='macro'))

('F1 Score KNN : ', 0.6520056964547184)
('Accuracy Score KNN : ', 0.6233254531126872)
('Precision Score KNN : ', 0.7035398004343844)
('Recall Score KNN : ', 0.7820851291934147)


In [18]:
tree = DecisionTreeClassifier()

tree.fit(x_train, y_train)

tree_pred = tree.predict(x_test)
print('F1 Score DTL : ', f1_score(tree_pred, y_test, average='macro'))
print('Accuracy Score DTL : ', accuracy_score(tree_pred, y_test))
print('Precision Score DTL : ', precision_score(tree_pred, y_test, average='macro'))
print('Recall Score DTL : ', recall_score(tree_pred, y_test, average='macro'))

('F1 Score DTL : ', 0.8258705093531823)
('Accuracy Score DTL : ', 0.7880220646178093)
('Precision Score DTL : ', 0.8174273718179443)
('Recall Score DTL : ', 0.835596178720873)


In [19]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
svc = LinearSVC()

svc.fit(x_train, y_train)
svc_pred = svc.predict(x_test)

print('F1 Score SVM : ', f1_score(svc_pred, y_test, average='macro'))
print('Accuracy Score SVM : ', accuracy_score(svc_pred, y_test))
print('Precision Score SVM : ', precision_score(svc_pred, y_test, average='macro'))
print('Recall Score SVM : ', recall_score(svc_pred, y_test, average='macro'))

('F1 Score SVM : ', 0.8733553896418226)
('Accuracy Score SVM : ', 0.8486997635933806)
('Precision Score SVM : ', 0.8698415741890054)
('Recall Score SVM : ', 0.8776318719116382)


In [21]:
x_test

<1269x11404 sparse matrix of type '<type 'numpy.int64'>'
	with 15328 stored elements in Compressed Sparse Row format>

In [30]:
pred

array([0])

In [58]:
joblib.dump(svc, 'svm-model.joblib')

['svm-model.joblib']